In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
application_df.columns


Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [12]:
# Define the new, smaller model architecture

# The number of input features is still the number of columns in the scaled training data
number_input_features = X_train_scaled.shape[1]  # This gives the number of features

# Number of nodes for each hidden layer (experimenting with smaller values)
hidden_nodes_layer1 = 8   # Smaller number of nodes in the first hidden layer
hidden_nodes_layer2 = 4   # Smaller number of nodes in the second hidden layer

# Create the neural network model using the Sequential API
nn = tf.keras.models.Sequential()

# First hidden layer (Smaller number of nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer (Smaller number of nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer - Since this is a binary classification problem, we use sigmoid activation
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


c:\Users\zilan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 8)              │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 937 (3.66 KB)

 Trainable params: 937 (3.66 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:

# Compile the model
nn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

# Implementing EarlyStopping to stop training once the validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

# ModelCheckpoint to save the best model during training
model_checkpoint = ModelCheckpoint("AlphabetSoupCharity_Optimization.h5",
                                   save_best_only=True,
                                   monitor='val_loss',
                                   mode='min',
                                   verbose=1)

# Train the model with EarlyStopping and ModelCheckpoint callbacks
fit_model = nn.fit(X_train_scaled,            # Training features
                   y_train,                  # Training labels
                   epochs=100,               # Number of epochs
                   validation_split=0.2,     # Split 20% for validation
                   verbose=1,                # Show progress
                   callbacks=[early_stopping, model_checkpoint])  # Add callbacks

# Evaluate the model's performance on the test set
test_loss, test_acc = nn.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Epoch 1/100
679/686 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8059 - loss: 0.4401
Epoch 1: val_loss improved from inf to 0.45678, saving model to AlphabetSoupCharity_Optimization.h5


686/686 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.8059 - loss: 0.4397 - val_accuracy: 0.7985 - val_loss: 0.4568
Epoch 2/100
682/686 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8860 - loss: 0.2814
Epoch 2: val_loss did not improve from 0.45678
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8860 - loss: 0.2813 - val_accuracy: 0.5751 - val_loss: 1.2323
Epoch 3/100
682/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9429 - loss: 0.1495
Epoch 3: val_loss did not improve from 0.45678
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9429 - loss: 0.1495 - val_accuracy: 0.5838 - val_loss: 0.9657
Epoch 4/100
683/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9577 - loss: 0.1025
Epoch 4: val_loss did not improve from 0.45678
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9577 - loss: 0.1026 - val_accuracy: 0.5809 - val_loss: 1.4944
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9608 - loss: 0.0941
Epoch 5: val_loss did not improve from 0.

In [ ]:


# The number of input features is still the number of columns in the scaled training data
number_input_features = X_train_scaled.shape[1]  # This gives the number of features

# Number of nodes for each hidden layer (experimenting with different values)
hidden_nodes_layer1 = 16  # Increased nodes in the first hidden layer
hidden_nodes_layer2 = 8   # Decreased nodes in the second hidden layer
hidden_nodes_layer3 = 4   # Added an additional hidden layer with 4 nodes

# Create the neural network model using the Sequential API
nn = tf.keras.models.Sequential()

# First hidden layer (Increased nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer (Decreased nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer (New layer with 4 nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer - Since this is a binary classification problem, we use sigmoid activation
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,969 (7.69 KB)

 Trainable params: 1,969 (7.69 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Separate the categorical and numerical columns
categorical_cols = application_df.select_dtypes(include=["object"]).columns
numerical_cols = application_df.select_dtypes(exclude=["object"]).columns

# Apply one-hot encoding to categorical columns
application_df_encoded = pd.get_dummies(application_df, columns=categorical_cols)

# Separate the target variable and the features
X = application_df_encoded.drop("IS_SUCCESSFUL", axis=1)
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer: Success (0 or 1)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model's performance on the test set
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Epoch 1/20


c:\Users\zilan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6991 - loss: 0.5979 - val_accuracy: 0.7201 - val_loss: 0.5739
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7297 - loss: 0.5577 - val_accuracy: 0.7261 - val_loss: 0.5679
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7233 - loss: 0.5607 - val_accuracy: 0.7267 - val_loss: 0.5657
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7292 - loss: 0.5587 - val_accuracy: 0.7270 - val_loss: 0.5647
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7316 - loss: 0.5519 - val_accuracy: 0.7235 - val_loss: 0.5633
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7270 - loss: 0.5586 - val_accuracy: 0.7287 - val_loss: 0.5640
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7300 - loss: 0.5531 - val_accuracy: 0.7273 - val_loss: 0.5638
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7265 - loss: 0.5543 - val_accuracy: 0.7224 - val_

In [10]:
# Define the new model architecture with more layers and nodes

# The number of input features is still the number of columns in the scaled training data
number_input_features = X_train_scaled.shape[1]  # This gives the number of features

# Number of nodes for each hidden layer (experimenting with different values)
hidden_nodes_layer1 = 64  # Increased nodes in the first hidden layer
hidden_nodes_layer2 = 32  # Increased nodes in the second hidden layer
hidden_nodes_layer3 = 16  # Increased nodes in the third hidden layer
hidden_nodes_layer4 = 8   # A fourth hidden layer with fewer nodes

# Create the neural network model using the Sequential API
nn = tf.keras.models.Sequential()

# First hidden layer (Increased nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer (Increased nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer (Increased nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer (Reduced nodes)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer - Since this is a binary classification problem, we use sigmoid activation
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 64)             │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,921 (38.75 KB)

 Trainable params: 9,921 (38.75 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:

# Compile the model (ensure this is done before training)
nn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

# Implementing EarlyStopping to stop training once the validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

# ModelCheckpoint to save the best model during training
model_checkpoint = ModelCheckpoint("AlphabetSoupCharity_Optimization.h5",
                                   save_best_only=True,
                                   monitor='val_loss',
                                   mode='min',
                                   verbose=1)

# Train the model with EarlyStopping and ModelCheckpoint callbacks
fit_model = nn.fit(X_train_scaled,            # Training features
                   y_train,                  # Training labels
                   epochs=100,               # Number of epochs
                   validation_split=0.2,     # Split 20% for validation
                   verbose=1,                # Show progress
                   callbacks=[early_stopping, model_checkpoint])  # Add callbacks

# Evaluate the model's performance on the test set
test_loss, test_acc = nn.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


Epoch 1/100
675/686 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8877 - loss: 0.2861
Epoch 1: val_loss improved from inf to 0.44059, saving model to AlphabetSoupCharity_Optimization.h5


686/686 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8877 - loss: 0.2860 - val_accuracy: 0.7999 - val_loss: 0.4406
Epoch 2/100
685/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9284 - loss: 0.1786
Epoch 2: val_loss did not improve from 0.44059
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9284 - loss: 0.1786 - val_accuracy: 0.5867 - val_loss: 0.8908
Epoch 3/100
680/686 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9563 - loss: 0.1117
Epoch 3: val_loss did not improve from 0.44059
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9563 - loss: 0.1117 - val_accuracy: 0.5882 - val_loss: 1.1158
Epoch 4/100
681/686 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9626 - loss: 0.0923
Epoch 4: val_loss did not improve from 0.44059
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9626 - loss: 0.0924 - val_accuracy: 0.5866 - val_loss: 1.2771
Epoch 5/100
680/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9635 - loss: 0.0898
Epoch 5: val_loss did not improve from 0.